In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

In [2]:
# Part 1 - Data Preprocessing

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [5]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/fepas/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/fepas/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# Part 2 - Now let's make the ANN!

In [9]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [10]:
# Initialising the ANN
classifier = Sequential()

In [11]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
8000/8000 [==============================] - 3s 399us/step - loss: 0.4879 - acc: 0.7951
Epoch 2/10
8000/8000 [==============================] - 1s 70us/step - loss: 0.4285 - acc: 0.7960
Epoch 3/10
8000/8000 [==============================] - 1s 79us/step - loss: 0.4225 - acc: 0.7960
Epoch 4/10
8000/8000 [==============================] - 1s 71us/step - loss: 0.4181 - acc: 0.8214
Epoch 5/10
8000/8000 [==============================] - 1s 70us/step - loss: 0.4154 - acc: 0.8299
Epoch 6/10
8000/8000 [==============================] - 1s 71us/step - loss: 0.4137 - acc: 0.8322
Epoch 7/10
8000/8000 [==============================] - 1s 69us/step - loss: 0.4120 - acc: 0.8325
Epoch 8/10
8000/8000 [==============================] - 1s 69us/step - loss: 0.4106 - acc: 0.8342
Epoch 9/10
8000/8000 [==============================] - 1s 69us/step - loss: 0.4094 - acc: 0.83

In [12]:
# Part 3 - Making predictions and evaluating the model

In [13]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [14]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [15]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [16]:
print(cm)

[[1559   36]
 [ 284  121]]


In [17]:
# Part 4 - Evaluating, Improving and Tuning the ANN

In [18]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [19]:
print(accuracies)
print(mean)
print(variance)

[0.825      0.81999999 0.81875    0.7825     0.84624999 0.82625
 0.83625    0.81       0.79875    0.83124999]
0.8194999955222011
0.017608590461640705


In [20]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [21]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [22]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [23]:
classifier = KerasClassifier(build_fn = build_classifier)

In [24]:
parameters = {'batch_size': [10, 25, 50],
              'epochs': [10, 25, 50],
              'optimizer': ['adam', 'rmsprop']}

In [25]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [26]:
grid_search = grid_search.fit(X_train, y_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/10
7200/7200 [==============================] - 1s 107us/step - loss: 0.4903 - acc: 0.7972
Epoch 2/10
7200/7200 [==============================] - 1s 75us/step - loss: 0.4306 - acc: 0.7971
Epoch 3/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4288 - acc: 0.7971
Epoch 4/10
7200/7200 [==============================] - 1s 80us/step - loss: 0.4267 - acc: 0.7992
Epoch 5/10
7200/7200 [==============================] - 1s 90us/step - loss: 0.4243 - acc: 0.8210
Epoch 6/10
7200/7200 [==============================] - 1s 83us/step - loss: 0.4238 - acc: 0.8254
Epoch 7/10
7200/7200 [==============================] - 1s 77us/step - loss: 0.4257 - acc: 0.8236
Epoch 8/10
7200/7200 [==============================] - 1s 76us/step - loss: 0.4287 - acc: 0.8244
Epoch 9/10
7200/7200 [==============================] - 1s 78us/step - loss: 0.4234 - acc: 0.8276
Ep

7200/7200 [==============================] - 1s 87us/step - loss: 0.4199 - acc: 0.8301
Epoch 1/10
7200/7200 [==============================] - 1s 168us/step - loss: 0.5053 - acc: 0.7958
Epoch 2/10
7200/7200 [==============================] - 1s 85us/step - loss: 0.4399 - acc: 0.7957
Epoch 3/10
7200/7200 [==============================] - 1s 87us/step - loss: 0.4378 - acc: 0.7957
Epoch 4/10
7200/7200 [==============================] - 1s 89us/step - loss: 0.4381 - acc: 0.7957
Epoch 5/10
7200/7200 [==============================] - 1s 89us/step - loss: 0.4358 - acc: 0.7957
Epoch 6/10
7200/7200 [==============================] - 1s 105us/step - loss: 0.4366 - acc: 0.7957
Epoch 7/10
7200/7200 [==============================] - 1s 87us/step - loss: 0.4312 - acc: 0.7982
Epoch 8/10
7200/7200 [==============================] - 1s 84us/step - loss: 0.4321 - acc: 0.8178
Epoch 9/10
7200/7200 [==============================] - 1s 97us/step - loss: 0.4259 - acc: 0.8233
Epoch 10/10
7200/7200 [======

7200/7200 [==============================] - 1s 88us/step - loss: 0.4361 - acc: 0.7969
Epoch 4/10
7200/7200 [==============================] - 1s 86us/step - loss: 0.4312 - acc: 0.7969
Epoch 5/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4295 - acc: 0.7969
Epoch 6/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4308 - acc: 0.7969
Epoch 7/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4295 - acc: 0.7969
Epoch 8/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4299 - acc: 0.8067
Epoch 9/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4277 - acc: 0.8229
Epoch 10/10
7200/7200 [==============================] - 1s 88us/step - loss: 0.4279 - acc: 0.8251
Epoch 1/10
7200/7200 [==============================] - 2s 217us/step - loss: 0.5088 - acc: 0.7961
Epoch 2/10
7200/7200 [==============================] - 1s 89us/step - loss: 0.4360 - acc: 0.7962
Epoch 3/10
7200/7200 [=======

7200/7200 [==============================] - 1s 100us/step - loss: 0.4194 - acc: 0.8304
Epoch 22/25
7200/7200 [==============================] - 1s 100us/step - loss: 0.4226 - acc: 0.8283
Epoch 23/25
7200/7200 [==============================] - 1s 99us/step - loss: 0.4222 - acc: 0.8299
Epoch 24/25
7200/7200 [==============================] - 1s 99us/step - loss: 0.4216 - acc: 0.8315
Epoch 25/25
7200/7200 [==============================] - 1s 99us/step - loss: 0.4215 - acc: 0.8301
Epoch 1/25
7200/7200 [==============================] - 2s 267us/step - loss: 0.5099 - acc: 0.7950
Epoch 2/25
7200/7200 [==============================] - 1s 97us/step - loss: 0.4378 - acc: 0.7956
Epoch 3/25
7200/7200 [==============================] - 1s 99us/step - loss: 0.4336 - acc: 0.7956
Epoch 4/25
7200/7200 [==============================] - 1s 99us/step - loss: 0.4341 - acc: 0.7956
Epoch 5/25
7200/7200 [==============================] - 1s 101us/step - loss: 0.4331 - acc: 0.7956
Epoch 6/25
7200/7200 [=

7200/7200 [==============================] - 1s 106us/step - loss: 0.4350 - acc: 0.7944
Epoch 4/25
7200/7200 [==============================] - 1s 105us/step - loss: 0.4311 - acc: 0.7944
Epoch 5/25
7200/7200 [==============================] - 1s 104us/step - loss: 0.4317 - acc: 0.7944
Epoch 6/25
7200/7200 [==============================] - 1s 104us/step - loss: 0.4268 - acc: 0.8060
Epoch 7/25
7200/7200 [==============================] - 1s 104us/step - loss: 0.4279 - acc: 0.8175
Epoch 8/25
7200/7200 [==============================] - 1s 105us/step - loss: 0.4259 - acc: 0.8244
Epoch 9/25
7200/7200 [==============================] - 1s 108us/step - loss: 0.4299 - acc: 0.8256
Epoch 10/25
7200/7200 [==============================] - 1s 105us/step - loss: 0.4254 - acc: 0.8262
Epoch 11/25
7200/7200 [==============================] - 1s 105us/step - loss: 0.4271 - acc: 0.8272
Epoch 12/25
7200/7200 [==============================] - 1s 107us/step - loss: 0.4228 - acc: 0.8276
Epoch 13/25
7200/7

...

Epoch 39/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4184 - acc: 0.8296
Epoch 40/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4195 - acc: 0.8336
Epoch 41/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4184 - acc: 0.8283
Epoch 42/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4170 - acc: 0.8314
Epoch 43/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4176 - acc: 0.8322
Epoch 44/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4174 - acc: 0.8317
Epoch 45/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4222 - acc: 0.8307
Epoch 46/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4184 - acc: 0.8290
Epoch 47/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4173 - acc: 0.8303
Epoch 48/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4200 - acc: 0.8303
Epoch 49/5

7200/7200 [==============================] - 1s 78us/step - loss: 0.4234 - acc: 0.8279
Epoch 22/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4239 - acc: 0.8283
Epoch 23/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4239 - acc: 0.8285
Epoch 24/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4239 - acc: 0.8289
Epoch 25/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4244 - acc: 0.8289
Epoch 26/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4207 - acc: 0.8286
Epoch 27/50
7200/7200 [==============================] - 1s 76us/step - loss: 0.4217 - acc: 0.8267
Epoch 28/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4214 - acc: 0.8304
Epoch 29/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4216 - acc: 0.8303
Epoch 30/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4241 - acc: 0.8296
Epoch 31/50
7200/7200 

Epoch 4/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4423 - acc: 0.7962
Epoch 5/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4354 - acc: 0.7962
Epoch 6/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4369 - acc: 0.7962
Epoch 7/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4332 - acc: 0.7962
Epoch 8/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4322 - acc: 0.7962
Epoch 9/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4297 - acc: 0.7962
Epoch 10/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4314 - acc: 0.7962
Epoch 11/50
7200/7200 [==============================] - 1s 77us/step - loss: 0.4285 - acc: 0.7962
Epoch 12/50
7200/7200 [==============================] - 1s 79us/step - loss: 0.4295 - acc: 0.7962
Epoch 13/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4280 - acc: 0.7962
Epoch 14/50
7200

7200/7200 [==============================] - 1s 79us/step - loss: 0.4187 - acc: 0.8306
Epoch 37/50
7200/7200 [==============================] - 1s 80us/step - loss: 0.4199 - acc: 0.8310
Epoch 38/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4206 - acc: 0.8308
Epoch 39/50
7200/7200 [==============================] - 1s 80us/step - loss: 0.4193 - acc: 0.8321
Epoch 40/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4215 - acc: 0.8296
Epoch 41/50
7200/7200 [==============================] - 1s 79us/step - loss: 0.4222 - acc: 0.8322
Epoch 42/50
7200/7200 [==============================] - 1s 79us/step - loss: 0.4188 - acc: 0.8299
Epoch 43/50
7200/7200 [==============================] - 1s 80us/step - loss: 0.4184 - acc: 0.8303
Epoch 44/50
7200/7200 [==============================] - 1s 80us/step - loss: 0.4197 - acc: 0.8282
Epoch 45/50
7200/7200 [==============================] - 1s 78us/step - loss: 0.4221 - acc: 0.8325
Epoch 46/50
7200/7200 

8000/8000 [==============================] - 1s 161us/step - loss: 0.4245 - acc: 0.8265
Epoch 19/50
8000/8000 [==============================] - 1s 160us/step - loss: 0.4206 - acc: 0.8302
Epoch 20/50
8000/8000 [==============================] - 1s 160us/step - loss: 0.4204 - acc: 0.8282
Epoch 21/50
8000/8000 [==============================] - 1s 160us/step - loss: 0.4244 - acc: 0.8289
Epoch 22/50
8000/8000 [==============================] - 1s 159us/step - loss: 0.4252 - acc: 0.8306
Epoch 23/50
8000/8000 [==============================] - 1s 160us/step - loss: 0.4222 - acc: 0.8296
Epoch 24/50
8000/8000 [==============================] - 1s 159us/step - loss: 0.4203 - acc: 0.8302
Epoch 25/50
8000/8000 [==============================] - 1s 159us/step - loss: 0.4235 - acc: 0.8310
Epoch 26/50
8000/8000 [==============================] - 1s 159us/step - loss: 0.4215 - acc: 0.8294
Epoch 27/50
8000/8000 [==============================] - 1s 160us/step - loss: 0.4244 - acc: 0.8319
Epoch 28/50


In [27]:
best_parameters = grid_search.best_params_

In [30]:
best_accuracy = grid_search.best_score_

In [28]:
print(best_parameters)

{'batch_size': 25, 'epochs': 50, 'optimizer': 'adam'}


In [32]:
print(best_accuracy)

0.839125
